# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set_style('darkgrid')
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [32]:
df = pd.read_csv('homepage_actions.csv')
print("Unique IDs: ", df.id.unique().shape[0])
df.head()

Unique IDs:  6328


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [42]:
#how many people viewed and clicked
a = df.groupby('id').count().action
print("Viewed and Clicked: ", sum(a == 2))

#Did anybody click who didnt view
click = df[df.action == "click"]
print("Clicked: ", len(click))

Viewed and Clicked:  1860
Clicked:  1860


In [61]:
control = df[df.group=="control"]
experiment = df[df.group=="experiment"]


In [62]:
len(experiment.id.unique()) + len(control.id.unique()) 
#the combined unique ids from control and experiment equals
#the total unique ids thus there is no overlap

6328

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [76]:
con = control.pivot(index='id', columns='action', values='count')
con.fillna(value=0, inplace=True)

exp = experiment.pivot(index='id', columns='action', values='count')
exp.fillna(value=0, inplace=True)

In [81]:
con.click.mean()

0.2797118847539016

In [82]:
exp.click.mean()

0.3097463284379172

In [83]:
p = stats.ttest_ind(con.click, exp.click, equal_var=False)[1] / 2
p

0.004466402814337101

In [84]:
"""
p value is less than 0.05 thus the experimental
homepage was more effective
"""

'\np value is less than 0.05 thus the experimental\nhomepage was more effective\n'

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [87]:
con_click_rate = con.click.sum() / len(con.click)
con_click_rate

0.2797118847539016

In [88]:
exp_expected_clicks = len(exp.click)*con_click_rate
exp_expected_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [93]:
var = len(exp.click) * con_click_rate*(1 - con_click_rate)
std = np.sqrt(var)
std

24.568547907005815

In [94]:
z = (exp.click.sum() - exp_expected_clicks) / std
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [95]:
p = 1 - stats.norm.cdf(z)
p

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Yes, both are below the alpha of 0.05, so we can reject the null hypothesis in both cases**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.